# Things to do to improve performance.

So the first submission didn't score as well as the train set, not surprising!

Lets try and;

1. Extract more of the text from the html, the body and title aren't enough it seems.
2. Over/undersample and see if I improve performance with either strategy.

In [13]:
import numpy as np
import pandas as pd
import os
from bs4 import BeautifulSoup
import nltk
from nltk import wordpunct_tokenize
from nltk.stem.snowball import EnglishStemmer
from nltk.corpus import stopwords
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.decomposition import TruncatedSVD
from sklearn.pipeline import make_pipeline, make_union
from tpot.builtins import StackingEstimator
from sklearn.ensemble import ExtraTreesClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import FunctionTransformer
from sklearn.model_selection import train_test_split
import tldextract
from copy import copy

data_dir = "../data/2018-08-10_AV_Innoplexus/"

In [2]:
source_df = pd.read_csv(data_dir+'train_df_all_tokens.csv')
source_df.sample(10)

,Webpage_id,all_tokens,Tag
20504,30866,"radiat, oncolog, journalroj, orgroj, org, auth...",others
44858,66938,"gabriell, sulzberg, teva, pharmaceut, industri...",profile
46347,69061,"niph, clinic, trial, searchregist, date, basic...",clinicalTrials
16020,23948,"abstract, tr, scopolamin, hydrobromid, trihydr...",publication
11065,16337,"clinic, demograph, featur, patient, upper, gas...",conferences
50545,74928,"atyp, cellular, chorangioma, archiv, patholog,...",publication
19948,29959,"busi, champion, support, credenti, completfunc...",others
18978,28620,"light, bodi, dr, pillai, light, bodi, activ, p...",others
48718,72031,"role, surgeri, follow, incomplet, respons, hig...",thesis
4683,7039,"rna, interfer, target, carbohydr, sulfotransfe...",publication


Lets transform! And then train.

In [3]:
vectorizer = TfidfVectorizer(input='content', analyzer='word')
svd = TruncatedSVD(n_components=500, n_iter=5, random_state=27)

In [4]:
word_vector_array = vectorizer.fit_transform(source_df['all_tokens'])

In [5]:
svd_array = svd.fit_transform(word_vector_array)

In [7]:
vector_features = vectorizer.get_feature_names()
eigen_features = [vector_features[i] for i in svd.components_[0].argsort()[::-1]][:500]

train_df = pd.DataFrame(svd_array,columns=eigen_features)
train_df['Tag'] = source_df['Tag']
    
tags = train_df['Tag'].unique().tolist()
tags.sort()

tag_dict = {key: value for (key, value) in zip(tags,range(len(tags)))}
tag_dict_inverted = {v: k for k, v in tag_dict.items()}

train_df['Tag_encoded'] = train_df['Tag'].map(tag_dict)
train_df = train_df.drop('Tag',axis=1)

In [8]:
train_df.sample(10)

,css,modul,site,highwir,function,js,theme,var,contrib,return,...,poster,take,octob,network,calendar,ajax,lung,articlepubmedgoogl,food,Tag_encoded
3316,0.092541,-0.089419,-0.059649,-0.065743,0.020413,-0.055379,-0.030471,-0.003668,0.057126,-0.058498,...,-0.021577,0.003490,-0.030992,0.020668,-0.010333,-0.011474,-0.026304,0.008943,0.045523,0
25917,0.115486,-0.102599,-0.076677,-0.045482,0.032388,-0.044053,0.114816,0.004561,0.007806,0.062805,...,0.000383,0.000465,-0.003220,-0.002379,-0.000459,-0.002032,0.002598,-0.008014,0.004030,6
26955,0.108589,-0.065628,0.079823,-0.115073,-0.093317,0.081756,-0.014825,-0.014626,-0.018245,0.013540,...,-0.005135,-0.001819,-0.003505,-0.011493,0.004085,-0.008207,0.001956,0.006665,-0.011630,5
169,0.078730,-0.074454,-0.051153,-0.045399,0.021950,-0.042080,-0.015144,0.016944,0.011618,-0.049232,...,-0.015157,-0.000046,-0.002288,-0.012215,0.014362,0.011383,0.010977,-0.010017,-0.004404,0
34896,0.270064,-0.256366,-0.290990,-0.106143,0.064276,-0.094673,0.300458,-0.120056,-0.079729,0.150074,...,-0.001730,0.010091,0.003231,0.011901,0.016730,-0.004070,0.003269,-0.001560,-0.014020,5
35282,0.292288,-0.207929,-0.028868,-0.167748,0.000200,-0.068191,-0.136420,0.054615,-0.010918,-0.024327,...,0.009828,-0.010404,0.017115,0.031721,-0.020840,0.031672,-0.018661,0.019730,-0.030820,5
10747,0.126136,-0.098364,-0.070314,-0.001698,0.031128,-0.049684,-0.084914,-0.017293,-0.010185,0.008763,...,0.006247,0.003080,0.008264,-0.003796,-0.003045,0.006825,0.001916,-0.004134,-0.004863,1
51051,0.188078,-0.132807,-0.063273,-0.106782,0.047139,-0.076446,0.001285,-0.049738,0.119005,0.004365,...,-0.027278,0.002061,-0.012345,-0.011493,0.047575,0.081047,-0.029479,-0.033235,0.035247,4
17253,0.192551,-0.130858,0.181779,-0.268256,-0.233328,0.224201,-0.012207,-0.077884,-0.154232,0.026866,...,0.011414,0.010817,-0.006010,0.000753,-0.001922,0.010697,-0.000479,0.001730,0.011285,4
22547,0.071832,-0.045437,0.012433,-0.046974,-0.040424,0.008675,-0.022257,0.023930,0.098757,-0.003955,...,-0.015533,0.000545,0.002891,0.010531,0.001957,0.001508,-0.000395,0.018442,0.008237,2


In [19]:
exported_pipeline = make_pipeline(
    make_union(
        FunctionTransformer(copy),
        StackingEstimator(estimator=LogisticRegression(C=0.001, dual=True, penalty="l2"))
    ),
    ExtraTreesClassifier(
        bootstrap=False, criterion="entropy",
        max_features=0.35000000000000003, min_samples_leaf=1,
        min_samples_split=2, n_estimators=100, verbose=2,
        n_jobs=2
    )
)

In [15]:
x_cols = [x for x in train_df.columns if x != "Tag_encoded"]
X_train, X_test, y_train, y_test = train_test_split(
    train_df[x_cols],
    train_df['Tag_encoded'],
    test_size=0.33
)

In [20]:
print("Fitting the model...")
exported_pipeline.fit(X_train, y_train)
print("Done!")

Fitting the model...
building tree 1 of 100building tree 2 of 100

building tree 3 of 100
building tree 4 of 100
building tree 5 of 100
building tree 6 of 100
building tree 7 of 100building tree 8 of 100

building tree 9 of 100
building tree 10 of 100
building tree 11 of 100
building tree 12 of 100
building tree 13 of 100
building tree 14 of 100
building tree 15 of 100
building tree 16 of 100
building tree 17 of 100
building tree 18 of 100
building tree 19 of 100
building tree 20 of 100
building tree 21 of 100
building tree 22 of 100
building tree 23 of 100
building tree 24 of 100
building tree 25 of 100
building tree 26 of 100
building tree 27 of 100
building tree 28 of 100
building tree 29 of 100
building tree 30 of 100
building tree 31 of 100
building tree 32 of 100
building tree 33 of 100
building tree 34 of 100
building tree 35 of 100
building tree 36 of 100
building tree 37 of 100
building tree 38 of 100
building tree 39 of 100

[Parallel(n_jobs=2)]: Done  37 tasks      | elapsed:  1.1min



building tree 40 of 100
building tree 41 of 100
building tree 42 of 100
building tree 43 of 100
building tree 44 of 100
building tree 45 of 100
building tree 46 of 100
building tree 47 of 100
building tree 48 of 100
building tree 49 of 100
building tree 50 of 100
building tree 51 of 100
building tree 52 of 100
building tree 53 of 100
building tree 54 of 100
building tree 55 of 100
building tree 56 of 100
building tree 57 of 100
building tree 58 of 100
building tree 59 of 100
building tree 60 of 100
building tree 61 of 100
building tree 62 of 100
building tree 63 of 100
building tree 64 of 100
building tree 65 of 100
building tree 66 of 100
building tree 67 of 100
building tree 68 of 100
building tree 69 of 100
building tree 70 of 100
building tree 71 of 100
building tree 72 of 100
building tree 73 of 100
building tree 74 of 100
building tree 75 of 100
building tree 76 of 100
building tree 77 of 100
building tree 78 of 100
building tree 79 of 100
building tree 80 of 100
building tree 8

[Parallel(n_jobs=2)]: Done 100 out of 100 | elapsed:  2.8min finished


Done!


In [21]:
#Sanity qual check
from sklearn.metrics import confusion_matrix, f1_score
preds = exported_pipeline.predict(X_test)
conf_mat = confusion_matrix(y_test, preds)

[Parallel(n_jobs=2)]: Done  37 tasks      | elapsed:    0.2s
[Parallel(n_jobs=2)]: Done 100 out of 100 | elapsed:    0.6s finished


In [22]:
conf_mat

array([[ 954,    0,    0,    0,    0,    2,    0,    1,    0],
       [   1, 1450,    0,    0,    7,   63,    2,   35,    0],
       [   0,    0, 1493,    0,    1,    6,    0,    1,    0],
       [   0,    7,    0,  368,    8,   25,    0,   35,    0],
       [   1,    7,    1,    7, 2362,  251,    4,   34,    0],
       [   1,   11,    9,    6,  164, 5372,   29,   46,    0],
       [   0,    2,    1,    0,    7,  103, 1592,   15,    0],
       [   4,   11,    0,    5,    7,   45,    1, 2436,    0],
       [   2,    0,    0,    0,    1,    5,    0,   16,  621]],
      dtype=int64)

In [23]:
model_f1 = f1_score(y_test, preds, average='weighted')
model_f1

0.9437544288440044

In [24]:
test_df = pd.read_csv(data_dir+"test_df_all_tokens.csv")

In [25]:
test_vector_arr = vectorizer.transform(test_df['all_tokens'])

In [26]:
test_svd_array = svd.transform(test_vector_arr)

In [27]:
test_df_predict = pd.DataFrame(test_svd_array,columns=eigen_features)
test_df_predict.head()

,css,modul,site,highwir,function,js,theme,var,contrib,return,...,cardiovascular,poster,take,octob,network,calendar,ajax,lung,articlepubmedgoogl,food
0,0.091652,-0.085779,-0.072362,-0.039760,0.005750,-0.039003,-0.000144,0.019785,0.036591,-0.019110,...,-0.028194,0.001056,-0.006351,0.033877,0.019432,0.016251,0.015640,0.020392,-0.004488,-0.003672
1,0.150894,-0.141412,-0.132741,-0.049788,0.018620,-0.070116,0.026077,0.014035,0.004262,-0.031105,...,0.012543,0.021684,-0.017101,0.005762,0.002776,-0.014640,0.000704,0.012322,0.001729,0.018138
2,0.132956,-0.127543,-0.113189,-0.045418,0.018606,-0.049239,-0.009355,0.042892,-0.000426,-0.035648,...,0.013711,0.016191,-0.013507,0.004946,0.003705,-0.008311,0.021261,0.016739,-0.000424,0.020296
3,0.163929,-0.154279,-0.151116,-0.061142,0.025371,-0.078909,0.000386,-0.024030,-0.008466,-0.070426,...,0.017696,0.006896,-0.008185,-0.014871,0.012218,-0.007919,0.001097,0.001967,-0.004248,0.014655
4,0.166741,-0.150701,-0.144927,-0.080986,0.014611,-0.080089,-0.003312,0.008242,0.048792,-0.064241,...,0.011261,0.001177,-0.019018,-0.016352,0.014829,-0.012317,-0.001134,0.029049,-0.022630,0.019334


In [28]:
submission_preds = exported_pipeline.predict(test_df_predict)

[Parallel(n_jobs=2)]: Done  37 tasks      | elapsed:    0.1s
[Parallel(n_jobs=2)]: Done 100 out of 100 | elapsed:    0.5s finished


In [29]:
submission_df = pd.DataFrame(data=submission_preds,columns=['Tag'],index=test_df_predict.index)
submission_df['Tag'] = submission_df['Tag'].map(tag_dict_inverted)
submission_df['Webpage_id'] = test_df['Webpage_id'].values.tolist()
col_order = ["Webpage_id","Tag"]
submission_df = submission_df[col_order]
submission_df.to_csv('submission_02.csv',index=False)
submission_df.head()

,Webpage_id,Tag
0,31,clinicalTrials
1,32,clinicalTrials
2,33,clinicalTrials
3,34,clinicalTrials
4,35,clinicalTrials
